In [7]:
import requests
from bs4 import BeautifulSoup, XMLParsedAsHTMLWarning
import pandas as pd
import unicodedata
import os
import warnings


warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

BASE = "https://data.sec.gov/submissions/"

#reusable header for everywhere so website allows you to pass without seeming a bot
HEADERS = {
    "User-Agent": "DataResearchBot/1.0 (contact: some@example.com)",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}

# first trying with hardcoded URL of Google's 10-K
# Now we want to see if it can work with any other SEC URL
# It should, so then we can allow the user to choose it, to input it
# Reusable for any SEC, for any company, for available years in the SEC archives
# we make it simple for the user, asking them for a company name and year then finding the corresponding URL
# proceed as before using that URL to pull consolidated financial statements

# query user
tkr = input("Provide company ticker:").strip()
yr = input("Provide year").strip()

# use data to find URL

# go from company ticker to CIK
# CIK = central index key
def get_cik_from_ticker(ticker: str) -> str:
    url = "https://www.sec.gov/files/company_tickers.json"
    data = requests.get(url, headers=HEADERS).json()
    for entry in data.values():
        if entry['ticker'].lower() == ticker.lower():
            cik = str(entry['cik_str']).zfill(10)
            print(f"Found CIK {cik} for ticker {ticker}")
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

#Helper to get 10-K filing URL for a year
def get_10k_url(ticker: str, year: str) -> str:
    cik = get_cik_from_ticker(ticker)
    json_link = f"{BASE}CIK{cik}.json"
    print(json_link)
    headers = {
    "User-Agent": "MyResearchBot/1.0 (contact: myemail@example.com)",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive"}
    resp = requests.get(json_link, headers=headers)
    resp.raise_for_status()
    if "application/json" not in resp.headers.get("Content-Type", ""):
        print("Unexpected content:", resp.text[:200])
        raise RuntimeError(f"Did not receive JSON from SEC for {url}")
    data = resp.json()
    forms = data['filings']['recent']
    
    filings = data.get("filings", {}).get("recent", {})
    forms = filings.get("form", [])
    dates = filings.get("filingDate", [])
    accessions = filings.get("accessionNumber", [])
    documents = filings.get("primaryDocument", [])
    
    for form, date, acc, doc in zip(forms, dates, accessions, documents):
        # Only check form type
        if form.startswith("10-K"):
            acc_no_dashes = acc.replace("-", "")
            filing_url = (
                f"https://www.sec.gov/Archives/edgar/data/"
                f"{int(cik)}/{acc_no_dashes}/{doc}"
            )
            print(f"Found 10-K filing for {ticker} {year}: {filing_url}")
            return filing_url  # return immediately

    # if no 10-K found at all
    raise ValueError(f"No 10-K filing found for {ticker} in {year}")

url = get_10k_url(tkr, yr)
print(url)

# 1: Load & parse HTML
resp = requests.get(url, headers=HEADERS)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

# 2: Find consolidated statement tables
# query user for the daesired data for extraction into xlsx
desired_data = input("Type of data to extract, ie tax provisions, gross margins, income")
all_tables = soup.find_all("table")
target_tables = []
for table in all_tables:
    text = table.get_text(" ", strip=True).lower()
    if desired_data in text:
        target_tables.append(table)

print(f"Found {len(target_tables)} matching tables.")

# Helper to clean rows
def extract_table(table):
    rows = []
    for row in table.find_all("tr"):
        cells = []
        for cell in row.find_all(["td", "th"]):
            txt = cell.get_text(" ", strip=True)
            if txt.endswith(","): txt=txt.rstrip(",")
            txt = txt.replace(u'\xa0', ' ')       # fix &nbsp;
            txt = unicodedata.normalize("NFKC", txt)
            if txt not in ["$", "%"]:  # skip lone symbols
                # Convert (90) into -90
                if txt.startswith("(") and txt.endswith(")"):
                    inner = txt[1:-1]  # strip parentheses
                    txt = "-" + inner
                cells.append(txt)
        if cells:
            # collapse duplicates: ["Revenue","Revenue","90,272"] -> ["Revenue","90,272"]
            cleaned = []
            for c in cells:
                if not cleaned or c != cleaned[-1]:
                    cleaned.append(c)
            rows.append(cleaned)
    first_cell = rows[0][0].strip().lower()
    # Normalize all text to lowercase for comparison
    first_row = [c.strip().lower() for c in rows[0]]
    first_col = [r[0].strip().lower() for r in rows if r]

    # Check if any cell in first row OR first column contains "page" or "index"
    if any("page" in c or "index" in c for c in first_row + first_col):
        return None
    
    return pd.DataFrame(rows) if (rows and len(rows) > 4 and max(len(r) for r in rows) > 2) else None



# Step 3: Take all target table and extract
if not target_tables:
    print("No matching tables found.")
# now, need to recursively extract and place each table found on a new csv
# make new worksheets in one workbook, one per extracted table 
else:
    # folder for files
    output_dir = "tables_output"
    # define folder to store files
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, "financial_statements.xlsx")

    #ExcelWriter saves all tables into one workbook

    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    
        for i, table_tag in enumerate(target_tables):
            raw_df = extract_table(table_tag)
    
            if raw_df is None:
                continue
        
            # Reload and clean
            df = raw_df.fillna("")
            df = df.drop_duplicates()                      # Remove duplicate rows
            df = df.loc[:, ~df.T.duplicated()]             # Remove duplicate columns
            
            df.to_excel(writer, sheet_name=f"table_{i}"[:31], index=False)
      
            print(f"Table {i} saved as sheet in {output_file}")

    print(f"All valid tables saved to {output_file}")


Provide company ticker: AAPL
Provide year 2021


Found CIK 0000320193 for ticker AAPL
https://data.sec.gov/submissions/CIK0000320193.json
Found 10-K filing for AAPL 2021: https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm
https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm


Type of data to extract, ie tax provisions, gross margins, income income


Found 16 matching tables.
Table 0 saved as sheet in tables_output/financial_statements.xlsx
Table 2 saved as sheet in tables_output/financial_statements.xlsx
Table 3 saved as sheet in tables_output/financial_statements.xlsx
Table 4 saved as sheet in tables_output/financial_statements.xlsx
Table 5 saved as sheet in tables_output/financial_statements.xlsx
Table 6 saved as sheet in tables_output/financial_statements.xlsx
Table 7 saved as sheet in tables_output/financial_statements.xlsx
Table 8 saved as sheet in tables_output/financial_statements.xlsx
Table 9 saved as sheet in tables_output/financial_statements.xlsx
Table 10 saved as sheet in tables_output/financial_statements.xlsx
Table 12 saved as sheet in tables_output/financial_statements.xlsx
Table 13 saved as sheet in tables_output/financial_statements.xlsx
All valid tables saved to tables_output/financial_statements.xlsx
